## Get Packages

In [40]:
using HTTP
using DataFrames
using JSON
using JSONStat
using DataStructures

## Get data from API

lets try to get data from Statistics Norway about salmon export: https://www.ssb.no/statbank/table/03024/

In [44]:
params = Dict("query" => [
        Dict(
            "code" => "Tid",
            "selection" => Dict(
                "filter" => "item",
                "values" => ["2022U34", "2022U35"]
            )
        )], "response" => Dict("format" => "json-stat2"))

Dict{String, Any} with 2 entries:
  "response" => Dict("format"=>"json-stat2")
  "query"    => Dict{String, Any}[Dict("selection"=>Dict{String, Any}("values"=…

In [45]:
url = "https://data.ssb.no/api/v0/no/table/03024/"

"https://data.ssb.no/api/v0/no/table/03024/"

In [52]:
r = HTTP.request("POST", url,
                 ["Content-Type" => "application/json"],
                 JSON.json(params))

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Cache-Control: private
Content-Type: application/json; charset=utf-8
Access-Control-Allow-Origin: *
Date: Wed, 25 Jan 2023 18:56:04 GMT
Content-Length: 1096
X-Varnish: 607171504
Age: 0
Via: 1.1 varnish (Varnish/6.0)
Content-Security-Policy: upgrade-insecure-requests
Expect-CT: enforce, max-age=3600
Referrer-Policy: no-referrer-when-downgrade
Strict-Transport-Security: max-age=86400
X-Content-Type-Options: nosniff
X-Frame-Options: SAMEORIGIN
X-XSS-Protection: 1; mode=block
X-Cache: MISS
X-UA-Compatible: IE=Edge
Accept-Ranges: bytes
Connection: keep-alive
Set-Cookie: ssbno=ffffffffc2a8b2e245525d5f4f58455e445a4a423660;expires=Wed, 25-Jan-2023 19:56:04 GMT;path=/;secure;httponly

{"class":"dataset","label":"03024: Eksport av oppalen laks, etter varegruppe, statistikkvariabel og uke","source":"Statistisk sentralbyrå","updated":"2023-01-25T07:00:00Z","id":["VareGrupper2","ContentsCode","Tid"],"size":[2,2,2],"dimension":{"VareGrupper2":{"extension":

In [53]:
bodyjs = JSON.parse(String(r.body), dicttype = DataStructures.OrderedDict)

OrderedDict{String, Any} with 11 entries:
  "class"     => "dataset"
  "label"     => "03024: Eksport av oppalen laks, etter varegruppe, statistikkv…
  "source"    => "Statistisk sentralbyrå"
  "updated"   => "2023-01-25T07:00:00Z"
  "id"        => Any["VareGrupper2", "ContentsCode", "Tid"]
  "size"      => Any[2, 2, 2]
  "dimension" => OrderedDict{String, Any}("VareGrupper2"=>OrderedDict{String, A…
  "value"     => Any[24160, 24010, 64.62, 61.94, 719, 585, 86.83, 87.02]
  "role"      => OrderedDict{String, Any}("time"=>Any["Tid"], "metric"=>Any["Co…
  "version"   => "2.0"
  "extension" => OrderedDict{String, Any}("px"=>OrderedDict{String, Any}("infof…

In [38]:
println(bodyjs)

Dict{String, Any} with 11 entries:
  "label"     => "03024: Eksport av oppalen laks, etter varegruppe, statistikkv…
  "role"      => Dict{String, Any}("time"=>Any["Tid"], "metric"=>Any["ContentsC…
  "id"        => Any["VareGrupper2", "ContentsCode", "Tid"]
  "value"     => Any[24160, 24010, 64.62, 61.94, 719, 585, 86.83, 87.02]
  "version"   => "2.0"
  "updated"   => "2023-01-25T07:00:00Z"
  "class"     => "dataset"
  "dimension" => Dict{String, Any}("ContentsCode"=>Dict{String, Any}("label"=>"…
  "size"      => Any[2, 2, 2]
  "extension" => Dict{String, Any}("px"=>Dict{String, Any}("infofile"=>"None", …
  "source"    => "Statistisk sentralbyrå"

In [57]:
salmon = readjsondataset(bodyjs)

OrderedDict{Any, Any} with 9 entries:
  "class"        => "dataset"
  "label"        => "03024: Eksport av oppalen laks, etter varegruppe, statisti…
  "source"       => "Statistisk sentralbyrå"
  "updated"      => "2023-01-25T07:00:00Z"
  "role"         => OrderedDict{String, Any}("time"=>Any["Tid"], "metric"=>Any[…
  "version"      => "2.0"
  "extension"    => OrderedDict{String, Any}("px"=>OrderedDict{String, Any}("in…
  "dimension"    => OrderedDict{String, OrderedDict{Any, Any}}("VareGrupper2"=>…
  "datasetframe" => 8×4 DataFrame…

In [61]:
println(salmon)

OrderedDict{Any, Any}("class" => "dataset", "label" => "03024: Eksport av oppalen laks, etter varegruppe, statistikkvariabel og uke", "source" => "Statistisk sentralbyrå", "updated" => "2023-01-25T07:00:00Z", "role" => OrderedDict{String, Any}("time" => Any["Tid"], "metric" => Any["ContentsCode"]), "version" => "2.0", "extension" => OrderedDict{String, Any}("px" => OrderedDict{String, Any}("infofile" => "None", "tableid" => "03024", "decimals" => 0)), "dimension" => OrderedDict{String, OrderedDict{Any, Any}}("VareGrupper2" => OrderedDict("category" => OrderedDict{Any, Any}("label" => OrderedDict{String, Any}("01" => "Fersk oppalen laks", "02" => "Frosen oppalen laks")), "label" => "varegruppe", "dimframe" => 2×1 DataFrame
 Row │ VareGrupper2 
     │ String       
─────┼──────────────
   1 │ 01
   2 │ 02), "ContentsCode" => OrderedDict("category" => OrderedDict{Any, Any}("label" => OrderedDict{String, Any}("Vekt" => "Vekt (tonn)", "Kilopris" => "Kilopris (kr)"), "unit" => OrderedDict{St

In [59]:
df = salmon["datasetframe"]

Row,VareGrupper2,ContentsCode,Tid,value
,String,String,String,Any
1,01,Vekt,2022U34,24160
2,01,Vekt,2022U35,24010
3,01,Kilopris,2022U34,64.62
4,01,Kilopris,2022U35,61.94
5,02,Vekt,2022U34,719
6,02,Vekt,2022U35,585
7,02,Kilopris,2022U34,86.83
8,02,Kilopris,2022U35,87.02


In [74]:
vareframe = labelframe("VareGrupper2", salmon)

Row,VareGrupper2,VareGrupper2_label
,String,Any
1,01,Fersk oppalen laks
2,02,Frosen oppalen laks


In [79]:
df = leftjoin(df, vareframe, on = :VareGrupper2)

Row,VareGrupper2,ContentsCode,Tid,value,VareGrupper2_label
,String,String,String,Any,Any
1,01,Vekt,2022U34,24160,Fersk oppalen laks
2,01,Vekt,2022U35,24010,Fersk oppalen laks
3,01,Kilopris,2022U34,64.62,Fersk oppalen laks
4,01,Kilopris,2022U35,61.94,Fersk oppalen laks
5,02,Vekt,2022U34,719,Frosen oppalen laks
6,02,Vekt,2022U35,585,Frosen oppalen laks
7,02,Kilopris,2022U34,86.83,Frosen oppalen laks
8,02,Kilopris,2022U35,87.02,Frosen oppalen laks
